In [1]:
#Federated Learning

#Using PySyft
import torch as th
import syft as sy

#Creating a hook
hook = sy.TorchHook(th)

W0709 22:25:19.290652 139649473939264 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.1-dev20190517). Fix this by compiling custom ops.
W0709 22:25:19.317587 139649473939264 deprecation_wrapper.py:119] From /home/ayush/anaconda3/lib/python3.7/site-packages/tf_encrypted/session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
#Creating a remote worker
bob = sy.VirtualWorker(hook, id='bob')

In [3]:
#To check objects associated with a worker
bob._objects

{}

In [4]:
#Sending data to a worker
x = th.tensor([1, 2, 3, 4, 5])
x = x.send(bob)

In [5]:
bob._objects

{70744674089: tensor([1, 2, 3, 4, 5])}

In [6]:
#Returns a pointer to remote object 
x

(Wrapper)>[PointerTensor | me:71856526140 -> bob:70744674089]

In [7]:
#Check the location where pointer is pointing
x.location

<VirtualWorker id:bob #tensors:1>

In [8]:
#Check the id of the worker
x.id_at_location

70744674089

In [9]:
#Check the id of the 
x.id

71856526140

In [10]:
#Check the owner info(in this case we are the owner)
x.owner

<VirtualWorker id:me #tensors:0>

In [11]:
#A local worker is created when PySyft is hooked to torch which sends instructions to other workers
hook.local_worker

<VirtualWorker id:me #tensors:0>

In [12]:
#Get information back from worker
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [13]:
bob._objects

{}

In [14]:
#Creating another virtual worker
alice = sy.VirtualWorker(hook, id='alice')

In [15]:
x = th.tensor([1, 2, 3, 4, 5])

In [16]:
bob._objects

{}

In [17]:
alice._objects

{}

In [18]:
x_ptr = x.send(bob, alice)

In [19]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:41525159493 -> bob:29161278184]
	-> (Wrapper)>[PointerTensor | me:68983282292 -> alice:51191542514]

In [20]:
bob._objects

{29161278184: tensor([1, 2, 3, 4, 5])}

In [21]:
alice._objects

{51191542514: tensor([1, 2, 3, 4, 5])}

In [22]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [23]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob, alice)

In [24]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

In [25]:
#Remote Arithematic
x = th.tensor([1, 2, 3, 4, 5]).send(bob)
y = th.tensor([1, 1, 1, 1, 1]).send(bob)

In [26]:
#Adding two tensors
z = x + y
z

(Wrapper)>[PointerTensor | me:42985299665 -> bob:37804492592]

In [27]:
z.get()

tensor([2, 3, 4, 5, 6])

In [28]:
#Adding two tensors
z = th.add(x, y)
z

(Wrapper)>[PointerTensor | me:12756137854 -> bob:701442217]

In [29]:
z.get()

tensor([2, 3, 4, 5, 6])

In [30]:
#Creating tensors with gradients on
x = th.tensor([1., 2, 3, 4, 5], requires_grad = True).send(bob)
y = th.tensor([1., 2, 3, 4, 5], requires_grad = True).send(bob)

In [31]:
#Calculating the sum of the tensors and the final sum of resulting tensor
z = (x + y).sum()
z

(Wrapper)>[PointerTensor | me:52863563496 -> bob:59549173274]

In [32]:
z.backward()

(Wrapper)>[PointerTensor | me:92111613081 -> bob:75708531370]

In [33]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [34]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [35]:
#Creating a Simple Linear Model
inputs = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad=True).send(bob)
targets = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [36]:
weights = th.tensor([[0.], [0]], requires_grad=True).send(bob)

In [37]:
#Creating a training loop
epochs = 10
for e in range(epochs):
    preds = inputs.mm(weights)
    loss = ((preds - targets)**2).sum()
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)  #Subtract gradient times learning rate from weights to update weights
    weights.grad *= 0
    
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


In [38]:
#Garbage collection
bob.clear_objects()

<VirtualWorker id:bob #tensors:0>

In [39]:
bob._objects

{}

In [40]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob)

In [41]:
bob._objects

{30378702174: tensor([1, 2, 3, 4, 5])}

In [42]:
del x  #Deleting the pointer to the tensor deletes the tensor

In [43]:
bob._objects

{}

In [44]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob)
x

(Wrapper)>[PointerTensor | me:82813550949 -> bob:99337235325]

In [45]:
x.child.garbage_collect_data  
#If this is set to off then deleting the pointer to the tensor would not delete the tensor itself

True

In [46]:
bob._objects

{99337235325: tensor([1, 2, 3, 4, 5])}

In [47]:
x = "abcd"
x

'abcd'

In [48]:
bob._objects  #This is a feature of jupyter notebook that the tensor did not get deleted

{99337235325: tensor([1, 2, 3, 4, 5])}

In [49]:
del x

In [50]:
bob._objects  #Tensor is still not deleted

{99337235325: tensor([1, 2, 3, 4, 5])}

In [51]:
#Creating a toy model with federated learning
from torch import nn, optim

In [52]:
# A Toy Dataset
data = th.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad=True)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True)

In [53]:
bob_data = data[0:2].send(bob)
bob_target = target[0:2].send(bob)

In [54]:
alice_data = data[2:4].send(alice)
alice_target = target[2:4].send(alice)

In [55]:
datasets = [(bob_data, bob_target), (alice_data, alice_target)]

In [56]:
def train(epochs = 20):
    
    model = nn.Linear(2, 1)
    optimizer = optim.SGD(model.parameters(), lr = 0.1)
    
    for e in range(epochs):
        
        for d, t in datasets:
            
            model = model.send(d.location)
            
            optimizer.zero_grad()
            
            preds = model(d)
            loss = ((preds - t)**2).sum()
            loss.backward()
            optimizer.step()
            
            model = model.get()
            
            print(loss.get().item())

In [57]:
train()

8.681060791015625
0.8603417277336121
0.6809864044189453
0.6389445066452026
0.314475953578949
0.3749649226665497
0.1794665902853012
0.21659626066684723
0.1040455549955368
0.12493225187063217
0.0604129284620285
0.07202529162168503
0.03509995713829994
0.041505105793476105
0.020406613126397133
0.023906586691737175
0.011873524636030197
0.01376350037753582
0.0069151767529547215
0.00792023353278637
0.004032074939459562
0.004555723629891872
0.0023543136194348335
0.00261948280967772
0.001377022359520197
0.0015057831769809127
0.0008070775656960905
0.000865518522914499
0.000474223168566823
0.0004975995398126543
0.0002794919419102371
0.0002862512192223221
0.0001653345680097118
0.00016486793174408376
9.824070002650842e-05
9.514878183836117e-05
5.8684530813479796e-05
5.5083964980440214e-05
3.5278426366858184e-05
3.203623782610521e-05


In [58]:
#Advanced remote execution
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [59]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob)
x

(Wrapper)>[PointerTensor | me:64971087766 -> bob:69611457054]

In [60]:
x = x.send(alice)
x

(Wrapper)>[PointerTensor | me:94000704620 -> alice:64971087766]

In [61]:
bob._objects

{69611457054: tensor([1, 2, 3, 4, 5])}

In [62]:
alice._objects

{64971087766: (Wrapper)>[PointerTensor | alice:64971087766 -> bob:69611457054]}

In [63]:
y = x + x
y

(Wrapper)>[PointerTensor | me:42133051126 -> alice:81718930169]

In [64]:
bob._objects

{69611457054: tensor([1, 2, 3, 4, 5]),
 80147966417: tensor([ 2,  4,  6,  8, 10])}

In [65]:
alice._objects

{64971087766: (Wrapper)>[PointerTensor | alice:64971087766 -> bob:69611457054],
 81718930169: (Wrapper)>[PointerTensor | alice:81718930169 -> bob:80147966417]}

In [66]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [67]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob).send(alice)

In [68]:
x

(Wrapper)>[PointerTensor | me:71293373465 -> alice:32685045978]

In [69]:
bob._objects

{97012026373: tensor([1, 2, 3, 4, 5])}

In [70]:
alice._objects

{32685045978: (Wrapper)>[PointerTensor | alice:32685045978 -> bob:97012026373]}

In [71]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:32685045978 -> bob:97012026373]

In [72]:
bob._objects

{97012026373: tensor([1, 2, 3, 4, 5])}

In [73]:
alice._objects

{}

In [74]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [75]:
bob._objects

{}

In [76]:
alice._objects

{}

In [78]:
#Pointer Chain operations
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [79]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob)

In [80]:
bob._objects

{95915836023: tensor([1, 2, 3, 4, 5])}

In [81]:
alice._objects

{}

In [82]:
#Moving a tensor from one worker to another
x.move(alice)

(Wrapper)>[PointerTensor | me:42563163857 -> alice:42563163857]

In [83]:
bob._objects

{}

In [84]:
alice._objects

{42563163857: tensor([1, 2, 3, 4, 5])}

In [85]:
x

(Wrapper)>[PointerTensor | me:42563163857 -> alice:42563163857]

In [86]:
x = th.tensor([1, 1, 1, 1, 1]).send(bob).send(alice)
x

(Wrapper)>[PointerTensor | me:82088057141 -> alice:67026670201]

In [87]:
bob._objects

{82973561926: tensor([1, 1, 1, 1, 1])}

In [88]:
alice._objects

{42563163857: tensor([1, 2, 3, 4, 5]),
 67026670201: (Wrapper)>[PointerTensor | alice:67026670201 -> bob:82973561926]}

In [89]:
#To transfer the tensor from bob to alice
x.remote_get()

(Wrapper)>[PointerTensor | me:82088057141 -> alice:67026670201]

In [90]:
bob._objects

{}

In [91]:
alice._objects

{42563163857: tensor([1, 2, 3, 4, 5]), 67026670201: tensor([1, 1, 1, 1, 1])}

In [92]:
x

(Wrapper)>[PointerTensor | me:82088057141 -> alice:67026670201]

In [93]:
#Move one tensor to bob
x.move(bob)

(Wrapper)>[PointerTensor | me:82088057141 -> bob:82088057141]

In [94]:
bob._objects

{82088057141: tensor([1, 1, 1, 1, 1])}

In [95]:
alice._objects

{42563163857: tensor([1, 2, 3, 4, 5])}